In [24]:
# -*- coding: utf-8 -*-
__author__ = 'nadya_motina'

import time
import codecs
import pandas
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.models import Word2Vec
from itertools import combinations

In [2]:
start_time = time.time()
model = Word2Vec.load_word2vec_format('../../Diplom/models/ruscorpora.model.bin', binary=True)
print 'Model loaded successfully. It took ', time.time() - start_time, ' seconds'

Model loaded successfully. It took  16.0365078449  seconds


In [3]:
infile = codecs.open('../Lyashevskaya/freqrnc2011.csv', 'r', 'utf8')
data = pandas.read_csv(infile, sep='\t')
infile.close()
data.head()

,Lemma,PoS,Freq(ipm),Doc
0,быть,v,12160.7,34184
1,год,s,3727.5,29477
2,мочь,v,2912.3,25413
3,человек,s,2723.0,20423
4,сказать,v,2396.6,15426


In [4]:
wg = nx.DiGraph()

for i in range(len(data)):
    pos = data['PoS'][i]
    ipm = float(data['Freq(ipm)'][i])
    lemma = (data['Lemma'][i] + '_' + pos.upper()).decode('utf8')
    try:
        vec = model[lemma]
        wg.add_node(lemma, ipm=ipm)
    except:
        pass
    
neighbors = wg.nodes(data=True)
print nx.info(wg)

Name: 
Type: DiGraph
Number of nodes: 43765
Number of edges: 0
Average in degree:   0.0000
Average out degree:   0.0000


In [40]:
start_time = time.time()

n = 0

for pair in combinations(neighbors, 2): # 957,665,730 combinations
    word0 = pair[0][0]
    word1 = pair[1][0]
    similarity = model.similarity(word0, word1)
    if similarity > 0:
        remaining = [vertex for vertex in neighbors if vertex != pair[0] and vertex != pair[1]]
        for vertex in remaining:
            drawedge = True
            candidate = vertex[0]
            try:
                if model.similarity(candidate, word1) > similarity and model.similarity(candidate, word0) > similarity:
                    drawedge = False
                    break
                if drawedge == True:
                    wg.add_edge(pair[0][0], pair[1][0], cos_sim=similarity)
            except:
                pass
    else:
        pass
    n += 1
    if n == 10000:
        break
    
print 'RNG built successfully. It took ', time.time() - start_time, ' seconds'
print nx.info(wg)

RNG built successfully. It took  185.005553961  seconds
Name: 
Type: DiGraph
Number of nodes: 43765
Number of edges: 5891
Average in degree:   0.1346
Average out degree:   0.1346
